# Yelp Sentiment Analysis

## Loading packages and importing data 

In [4]:
# Imports
import json
import typing
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier

[DUONG] RE: Dataset that we're using 
* I'm suggesting that we do NOT use checkin json. Only 2 columns of info on when the reviews were made, not really helpful for us. 
* 2 main datasets that we would be using is business and review. Potentially tip. Not sure about user, will have to figure that out. 

In [14]:
business=pd.read_json("C:/yelp/yelp_academic_dataset_business.json", lines=True)
review=pd.read_json("C:/yelp/yelp_academic_dataset_review.json", lines=True)

ValueError: Trailing data

[DUONG] RE: Fixing trailing data error
* Priyanka, not sure if it's showing on your end, but the json files are showing a trailing data error. 
* Trailing data basically means unexpected data or something is in an incorrect format
* This is not fixable by code, we have fix the json file itself. We need to talk to Prof F about this.
* Code to find out trailing data is below 


In [ ]:
file_path = "yelp_academic_dataset_business.json"

with open(file_path, "r") as json_file:
    try:
        data = json.load(json_file)
        # If the file is successfully parsed, it doesn't have trailing data
        print("JSON data is well-formed.")
    except json.JSONDecodeError as e:
        # JSONDecodeError occurs when there's a problem with the JSON structure
        print(f"JSON decode error: {e}")


In [6]:
business.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ..."
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,3.5,22,0,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Department Stores, Shopping, Fashion, Home & G...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ..."
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Brewpubs, Breweries, Food","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2..."


In [8]:
review.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11
1,BiTunyQ73aT9WBnpR9DZGw,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5,1,0,1,I've taken a lot of spin classes over the year...,2012-01-03 15:28:18
2,saUsX_uimxRlCVr67Z4Jig,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3,0,0,0,Family diner. Had the buffet. Eclectic assortm...,2014-02-05 20:30:30
3,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5,1,0,1,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03
4,Sx8TMOWLNuJBWer-0pcmoA,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4,1,0,1,Cute interior and owner (?) gave us tour of up...,2017-01-14 20:54:15


[DUONG] RE: Merging the data 
* I'm putting in code below to merge business and review, so that the reviews can be accompanied by the business name
* Depending on the NAs, we can adjust the type of join. For the time being, it's an outer join. 

In [ ]:
# Finding NAs in each column in business
missing_business = business.isna().sum()
print("Missing values in Business dataset:")
print(missing_business)

# Finding NAs in each column in review
missing_review = review.isna().sum()
print("Missing values in Review dataset:")
print(missing_review)

# business_id is the common key 
# Perform an outer join
business_review = business.merge(review, on='business_id', how='outer')

# Display the resulting DataFrame
print(business_review)

[DUONG] RE: Columns to keep and limiting our analysis scope

Business column keep list:
* business_id
* name
* city/state/postal code/longitude/latitude (to make map visualizations later maybe?)
* stars
* review count
* categories 

Review keep list:
* business_id
* text

Should we only look at restaurants, based on the size of this data? If you're down, see code below.

In [ ]:
# Filtering for restaurants
df_res = business_review[business_review['categories'].str.contains('Restaurant', case=False, na=False)]

# Keep only the desired columns
df_res = df_res[['whatever', 'columns', 'we', 'want', 'to', 'keep']]


In [10]:
tip=pd.read_json("C:/yelp/yelp_academic_dataset_tip.json", lines=True)



In [11]:
tip.head()

,user_id,business_id,text,date,compliment_count
0,AGNUgVwnZUey3gcPCJ76iw,3uLgwr0qeCNMjKenHJwPGQ,Avengers time with the ladies.,2012-05-18 02:17:21,0
1,NBN4MgHP9D3cw--SnauTkA,QoezRbYQncpRqyrLH6Iqjg,They have lots of good deserts and tasty cuban...,2013-02-05 18:35:10,0
2,-copOvldyKh1qr-vzkDEvw,MYoRNLb5chwjQe3c_k37Gg,It's open even when you think it isn't,2013-08-18 00:56:08,0
3,FjMQVZjSqY8syIO-53KFKw,hV-bABTK-glh5wj31ps_Jw,Very decent fried chicken,2017-06-27 23:05:38,0
4,ld0AperBXk1h6UbqmM80zw,_uN0OudeJ3Zl_tf6nxg5ww,Appetizers.. platter special for lunch,2012-10-06 19:43:09,0


In [13]:
user=pd.read_json("C:/yelp/yelp_academic_dataset_user.json", lines=True)
user.head()

,user_id,name,review_count,yelping_since,useful,funny,cool,elite,friends,fans,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,qVc8ODYU5SZjKXVBgXdI7w,Walker,585,2007-01-25 16:47:26,7217,1259,5994,2007,"NSCy54eWehBJyZdG2iE84w, pe42u7DcCH2QmI81NX-8qA...",267,...,65,55,56,18,232,844,467,467,239,180
1,j14WgRoU_-2ZE1aw1dXrJg,Daniel,4333,2009-01-25 04:35:42,43091,13066,27281,"2009,2010,2011,2012,2013,2014,2015,2016,2017,2...","ueRPE0CX75ePGMqOFVj6IQ, 52oH4DrRvzzl8wh5UXyU0A...",3138,...,264,184,157,251,1847,7054,3131,3131,1521,1946
2,2WnXYQFK0hXEoTxPtV2zvg,Steph,665,2008-07-25 10:41:00,2086,1010,1003,"2009,2010,2011,2012,2013","LuO3Bn4f3rlhyHIaNfTlnA, j9B4XdHUhDfTKVecyWQgyA...",52,...,13,10,17,3,66,96,119,119,35,18
3,SZDeASXq7o05mMNLshsdIA,Gwen,224,2005-11-29 04:38:33,512,330,299,"2009,2010,2011","enx1vVPnfdNUdPho6PH_wg, 4wOcvMLtU6a9Lslggq74Vg...",28,...,4,1,6,2,12,16,26,26,10,9
4,hA5lMy-EnncsH4JoR-hFGQ,Karen,79,2007-01-05 19:40:59,29,15,7,,"PBK4q9KEEBHhFvSXCUirIw, 3FWPpM7KU1gXeOM_ZbYMbA...",1,...,1,0,0,0,1,1,0,0,0,0


## Exploratory Data Analysis 

In [ ]:
# Assuming your DataFrame is named 'review' and the column with reviews is 'text'
df_res['review_length'] = df_res['text'].apply(len)

# Display statistics about review lengths
print("Review Length Statistics:")
print(df_res['review_length'].describe())

# Create a histogram to visualize the distribution of review lengths
plt.figure(figsize=(10, 6))
plt.hist(review['review_length'], bins=50, color='skyblue', edgecolor='black')
plt.title('Distribution of Review Lengths')
plt.xlabel('Review Length')
plt.ylabel('Number of Reviews')
plt.show()
# This should help us see skewness of distribution of review lengths

In [ ]:
# THIS CODE NEEDS ADJUSTMENT I JUST GOOGLED A REALLY SIMPLE EXAMPLE
# Sample text data
text_data = [
    "This is an example sentence.",
    "Another example sentence for tokenization.",
    "Word2Vec is a word embedding technique."
]

# Tokenize the text
tokenized_text = [word_tokenize(sentence.lower()) for sentence in text_data]

# Train a Word2Vec model
model = Word2Vec(tokenized_text, vector_size=100, window=5, min_count=1, sg=0)

# Get the word vector for a specific word
word_vector = model.wv['example']

# Find similar words
similar_words = model.wv.most_similar('example', topn=5)

# Display the word vector for 'example' and similar words
print("Word Vector for 'example':")
print(word_vector)
print("\nSimilar Words to 'example':")
for word, score in similar_words:
    print(f"{word}: {score}")


## Sample codes for potential next steps

[DUONG] Use machine learning techniques to identify exceptional words or phrases. We can consider:
* Feature selection techniques (e.g., SelectKBest) to select words that are most correlated with exceptional businesses.
* Classification models (e.g., Logistic Regression, Random Forest) to predict exceptional businesses based on the words used in the reviews.



In [ ]:
# Identifying exceptional outliers using Random Forest
X = tfidf_matrix
y = review_df['business_id'].isin(exceptional_businesses['business_id'])
clf = RandomForestClassifier()
clf.fit(X, y)

[DUONG] Examine the words or phrases with high feature importance scores or coefficients in the classification model to identify words associated with exceptional businesses.


In [ ]:
feature_importances = clf.feature_importances_

[DUONG] Create word clouds or bar charts to visualize the most common and exceptional words used in reviews for exceptional businesses.